<h1>Affordability</h1>

In [52]:
import sys
sys.path.insert(0, '../scripts/')

In [53]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from helper_functions import convert_census_to_postcode

import geopandas as gpd 
import folium

Obtaining property data

In [54]:
# Reading in median rental price per postcode
median_rent_df = pd.read_csv("../data/curated/median_rental_postcode.csv")
median_rent_df.head()

,Postcode,Cost
0,3000,520.0
1,3002,642.5
2,3003,520.0
3,3004,550.0
4,3006,551.5


In [55]:
# Reading in preprocessed property data
property_df = pd.read_csv("../data/curated/categorised_distances.csv")

# Select desired columns
property_df = property_df[["Cost", "Bed", "Bath", "Parking", "Postcode"]]
property_df.head()

,Cost,Bed,Bath,Parking,Postcode
0,440.0,1,1,0,3000
1,620.0,1,1,0,3000
2,300.0,1,1,0,3000
3,400.0,1,1,0,3000
4,625.0,2,2,1,3000


Calculating the average number of facilities

In [56]:
# Calculating average number of facilities per postcode
avg_bed = property_df.groupby(by = "Postcode")["Bed"].mean()
avg_bed.head()

Postcode
3000    1.601399
3002    1.909091
3003    1.796875
3004    1.771739
3006    1.785714
Name: Bed, dtype: float64

In [57]:
avg_bath = property_df.groupby(by = "Postcode")["Bath"].mean()
avg_bath.head()

Postcode
3000    1.300699
3002    1.318182
3003    1.343750
3004    1.423913
3006    1.469388
Name: Bath, dtype: float64

In [58]:
avg_parking = property_df.groupby(by = "Postcode")["Parking"].mean()
avg_parking.head()

Postcode
3000    0.356643
3002    1.000000
3003    0.609375
3004    1.097826
3006    0.647959
Name: Parking, dtype: float64

In [59]:
# Creating dataframe of average facilities per postcode
facilities_df = pd.DataFrame()
facilities_df["Postcode"] = property_df["Postcode"].unique()
facilities_df["Average # Beds"] = avg_bed.to_list()
facilities_df["Average # Baths"] = avg_bath.to_list()
facilities_df["Average # Parking"] = avg_parking.to_list()
facilities_df.head()

,Postcode,Average # Beds,Average # Baths,Average # Parking
0,3000,1.601399,1.300699,0.356643
1,3002,1.909091,1.318182,1.000000
2,3003,1.796875,1.343750,0.609375
3,3004,1.771739,1.423913,1.097826
4,3006,1.785714,1.469388,0.647959


In [60]:
# Joining median rental price per postcode with average facilities
facilities_df = facilities_df.set_index("Postcode")
price_by_facility_df = median_rent_df.join(facilities_df, on = "Postcode")
price_by_facility_df = price_by_facility_df.set_index("Postcode")
price_by_facility_df.head()

,Cost,Average # Beds,Average # Baths,Average # Parking
Postcode,,,,
3000,520.0,1.601399,1.300699,0.356643
3002,642.5,1.909091,1.318182,1.000000
3003,520.0,1.796875,1.343750,0.609375
3004,550.0,1.771739,1.423913,1.097826
3006,551.5,1.785714,1.469388,0.647959


Obtaining census/income data

In [61]:
# Reading in csv files for obtaining census data
sa2_postcode_map = pd.read_csv("../data/curated/sa2_postcode_mapping_2021.csv")
sa2_postcode_map.set_index("sa2_2021", inplace = True)
census_df = pd.read_csv("../data/curated/census_data.csv")

In [62]:
# Using helper function to convert sa2 mapping to postcodes
census_by_postcode_df = convert_census_to_postcode(census_df, sa2_postcode_map, "mean_no_zero")
census_by_postcode_df.head()

,postcode_2021,tot_population_11,tot_population_16,tot_population_21,avg_med_mortg_rep_11,avg_med_mortg_rep_16,avg_med_mortg_rep_21,avg_med_person_inc_11,avg_med_person_inc_16,avg_med_person_inc_21,avg_med_rent_16,avg_med_rent_11,avg_med_rent_21,avg_med_hh_inc_16,avg_med_hh_inc_11,avg_med_hh_inc_21,tot_avg_hh_size_16,tot_avg_hh_size_11,tot_avg_hh_size_21
0,3000,124551,167166,178424,2213.38,2040.38,2040.19,862.18,5483.82,6467.76,395.76,447.06,418.19,1482.53,1896.76,2159.41,1.88,1.97,1.86
1,3002,68729,82804,89023,2357.78,2173.67,2155.22,1091.80,8969.60,10432.90,398.00,460.33,449.67,1709.40,2415.00,2598.80,1.82,1.91,1.87
2,3003,15496,20633,23083,2200.00,2050.00,2085.00,701.50,716.00,1000.00,395.00,418.50,385.50,1466.00,1493.50,1751.00,2.15,2.15,1.95
3,3004,100879,123254,129273,2331.58,2155.67,2149.75,1066.08,7152.46,8339.46,391.15,446.83,440.75,1688.85,2270.46,2471.46,1.83,1.89,1.84
4,3006,21150,30239,36164,2477.25,2217.75,2079.00,1132.40,16783.00,19507.00,406.80,501.00,461.00,1637.20,2883.20,3088.80,1.80,1.92,1.92


In [63]:
# Removing unnecessary columns from census data (only need columns containing data from 2021)
census_by_postcode_df = census_by_postcode_df[["postcode_2021", "tot_population_21", "avg_med_mortg_rep_21", "avg_med_person_inc_21", "avg_med_rent_21", 
                            "avg_med_hh_inc_21", "tot_avg_hh_size_21"]]

# Selecting columns required for assessing affordability
census_income_df = census_by_postcode_df[["postcode_2021", "avg_med_mortg_rep_21", "avg_med_person_inc_21"]]
census_income_df.head()

,postcode_2021,avg_med_mortg_rep_21,avg_med_person_inc_21
0,3000,2040.19,6467.76
1,3002,2155.22,10432.90
2,3003,2085.00,1000.00
3,3004,2149.75,8339.46
4,3006,2079.00,19507.00


In [64]:
# Renaming postcode column of census dataframe and setting to index
census_income_df = census_income_df.rename({"postcode_2021": "Postcode"}, axis = 1)
census_income_df = census_income_df.set_index("Postcode")
census_income_df.head()

,avg_med_mortg_rep_21,avg_med_person_inc_21
Postcode,,
3000,2040.19,6467.76
3002,2155.22,10432.90
3003,2085.00,1000.00
3004,2149.75,8339.46
3006,2079.00,19507.00


In [65]:
# Joining census income dataframe to dataframe containg price & facilities information
df = census_income_df.join(price_by_facility_df, on = "Postcode", how = "right")
df.head()

,avg_med_mortg_rep_21,avg_med_person_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking
Postcode,,,,,,
3000,2040.19,6467.76,520.0,1.601399,1.300699,0.356643
3002,2155.22,10432.90,642.5,1.909091,1.318182,1.000000
3003,2085.00,1000.00,520.0,1.796875,1.343750,0.609375
3004,2149.75,8339.46,550.0,1.771739,1.423913,1.097826
3006,2079.00,19507.00,551.5,1.785714,1.469388,0.647959


Calculating affordability metric

In [66]:
# If the number of facilities is 0 (i.e. for number of bedrooms or number of parking spaces) replace 0 value with arbitrary small value epsilon
EPSILON = 10**(-6)
 
df.loc[df["Average # Beds"] == 0, "Average # Beds"] = EPSILON
df.loc[df["Average # Parking"] == 0, "Average # Parking"] = EPSILON

In [67]:
# Calculating weights for facilities (from previous correlation matrix know positive correlation exists between all 3 attributes and the cost)
bed_weight = df["Average # Beds"].corr(df["Cost"])
bath_weight = df["Average # Baths"].corr(df["Cost"])
parking_weight = df["Average # Parking"].corr(df["Cost"])

In [68]:
# Taking the median rental price divided by the median income (multiply by 100 to get cost as a percentage of income)
df["(Cost / Income)%"] = (df["Cost"] / df["avg_med_person_inc_21"]) * 100
df.head()

,avg_med_mortg_rep_21,avg_med_person_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking,(Cost / Income)%
Postcode,,,,,,,
3000,2040.19,6467.76,520.0,1.601399,1.300699,0.356643,8.039878
3002,2155.22,10432.90,642.5,1.909091,1.318182,1.000000,6.158403
3003,2085.00,1000.00,520.0,1.796875,1.343750,0.609375,52.000000
3004,2149.75,8339.46,550.0,1.771739,1.423913,1.097826,6.595151
3006,2079.00,19507.00,551.5,1.785714,1.469388,0.647959,2.827190


In [69]:
# Taking the median mortage divided by the median income (multiply by 100 to get mortage as a percentage of income)
df["(Mortage / Income)%"] = (df["avg_med_mortg_rep_21"] / df["avg_med_person_inc_21"]) * 100
df.head()

,avg_med_mortg_rep_21,avg_med_person_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking,(Cost / Income)%,(Mortage / Income)%
Postcode,,,,,,,,
3000,2040.19,6467.76,520.0,1.601399,1.300699,0.356643,8.039878,31.543997
3002,2155.22,10432.90,642.5,1.909091,1.318182,1.000000,6.158403,20.657919
3003,2085.00,1000.00,520.0,1.796875,1.343750,0.609375,52.000000,208.500000
3004,2149.75,8339.46,550.0,1.771739,1.423913,1.097826,6.595151,25.778048
3006,2079.00,19507.00,551.5,1.785714,1.469388,0.647959,2.827190,10.657713


In [70]:
# Calculating non-stadardised affordability metric
df["Affordability"] = ((bed_weight * df["Average # Beds"]) + (bath_weight * df["Average # Baths"]) + (parking_weight * df["Average # Parking"]) + 
                        df["(Cost / Income)%"] + df["(Mortage / Income)%"])
df.head()

,avg_med_mortg_rep_21,avg_med_person_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking,(Cost / Income)%,(Mortage / Income)%,Affordability
Postcode,,,,,,,,,
3000,2040.19,6467.76,520.0,1.601399,1.300699,0.356643,8.039878,31.543997,39.820298
3002,2155.22,10432.90,642.5,1.909091,1.318182,1.000000,6.158403,20.657919,27.006983
3003,2085.00,1000.00,520.0,1.796875,1.343750,0.609375,52.000000,208.500000,260.725464
3004,2149.75,8339.46,550.0,1.771739,1.423913,1.097826,6.595151,25.778048,32.580566
3006,2079.00,19507.00,551.5,1.785714,1.469388,0.647959,2.827190,10.657713,13.734440


In [71]:
# Calculating standardised affordability metric
min_affordability = df.sort_values(by = "Affordability").head(1)["Affordability"].tolist()[0]
#print(min_affordability)
max_affordability = df.sort_values(by = "Affordability", ascending = False).head(1)["Affordability"].tolist()[0]
#print(max_affordability)

df["Standardised Affordability"] = (df["Affordability"] - min_affordability) / (max_affordability - min_affordability)

# Taking 1 - standardised affordability (to make sure metric ranks 0 as least affordable & 1 as most affordable)
df["Standardised Affordability"] = (1 - df["Standardised Affordability"])

# Printing the top 10 most affordable postcodes
df.sort_values(by = "Standardised Affordability", ascending = False).head(10)

,avg_med_mortg_rep_21,avg_med_person_inc_21,Cost,Average # Beds,Average # Baths,Average # Parking,(Cost / Income)%,(Mortage / Income)%,Affordability,Standardised Affordability
Postcode,,,,,,,,,,
3006,2079.00,19507.00,551.5,1.785714,1.469388,0.647959,2.827190,10.657713,13.734440,1.000000
3002,2155.22,10432.90,642.5,1.909091,1.318182,1.000000,6.158403,20.657919,27.006983,0.968361
3004,2149.75,8339.46,550.0,1.771739,1.423913,1.097826,6.595151,25.778048,32.580566,0.955074
3000,2040.19,6467.76,520.0,1.601399,1.300699,0.356643,8.039878,31.543997,39.820298,0.937816
3141,2100.00,1398.33,530.0,1.820000,1.265000,0.865000,37.902355,150.179142,188.271440,0.583937
3400,1203.00,766.00,320.0,2.594595,1.270270,1.081081,41.775457,157.049608,198.993294,0.558378
3066,2167.00,1338.00,500.0,1.743590,1.179487,0.820513,37.369208,161.958146,199.503011,0.557163
3067,2167.00,1354.00,545.0,1.907407,1.240741,0.814815,40.251108,160.044313,200.483024,0.554826
3181,2066.00,1266.33,500.0,1.994286,1.234286,0.788571,39.484179,163.148626,202.820131,0.549255


In [72]:
# Outputting affordability csv 
df = df.reset_index()
affordability_df = df[["Postcode", "Standardised Affordability"]]
affordability_df.to_csv("../data/curated/affordability.csv")

Graphing standardised affordability

In [84]:
# Creating geoJSON file of postcode and geometry coordinates 
sf = gpd.read_file("../data/raw/external/POA_2021_AUST_GDA2020.shp")
postcodes = pd.read_csv("../data/raw/external/postcode.csv", names = ["POA_CODE21", "Name", "Area"])
postcodes["POA_CODE21"] = postcodes["POA_CODE21"].astype(int)

# Converting the geometry shaape to to latitude and longitude
# TAKEN FROM TUTE 2 NOTEBOOK
sf["geometry"] = sf["geometry"].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf = sf[sf["POA_CODE21"].between("3000", "4000")]
sf["POA_CODE21"] = sf["POA_CODE21"].astype(int)



gdf = gpd.GeoDataFrame(
    pd.merge(postcodes, sf, on = "POA_CODE21", how = "inner")
)

geoJSON = gdf[["POA_CODE21", "geometry"]].drop_duplicates("POA_CODE21").to_json()

In [85]:
# (y, x) since we want (lat, long)
gdf["centroid"] = gdf["geometry"].apply(lambda x: (x.centroid.y, x.centroid.x))
gdf[["Name", "POA_CODE21", "centroid"]].head()

affordability_df.rename({"Postcode": "POA_CODE21"})

m = folium.Map(location = [-37.8136, 144.9631], tiles = "Stamen Terrain", zoom_start = 10)

c = folium.Choropleth(
            geo_data = geoJSON, # geoJSON 
            name = "choropleth", # name of plot
            data = affordability_df, # data source
            columns = ["Postcode", "Standardised Affordability"], # the columns required
            key_on = "properties.POA_CODE21", # this is from the geoJSON's properties
            fill_color = "YlOrRd", # color scheme
            nan_fill_color = "grey",
            legend_name = "Affordability"
        )

c.add_to(m)
m.save(f"../plots/affordability_heatmap")

In [78]:
# Creating GeoPandas visualisation of top 10 most affordable postcodes
most_affordable_df = affordability_df.sort_values(by = "Standardised Affordability", ascending = False).head(10)

postcodes["POA_CODE21"] = postcodes["POA_CODE21"].astype(int)
postcodes = postcodes.loc[postcodes["POA_CODE21"].isin(most_affordable_df["Postcode"].to_list())]

In [81]:
most_affordable_df

,Postcode,Standardised Affordability
4,3006,1.000000
1,3002,0.968361
3,3004,0.955074
0,3000,0.937816
96,3141,0.583937
175,3400,0.558378
50,3066,0.557163
51,3067,0.554826
125,3181,0.549255
128,3184,0.545166


In [79]:
gdf = gpd.GeoDataFrame(
    pd.merge(postcodes, sf, on = "POA_CODE21", how = "inner")
)

geoJSON = gdf[["POA_CODE21", "geometry"]].drop_duplicates("POA_CODE21").to_json()

m = folium.Map(location = [-37.8136, 144.9631], tiles = "Stamen Terrain", zoom_start = 10)

m.add_child(folium.Choropleth(geo_data = geoJSON, name = "choropleth",))
m.save(f"../plots/most_affordable_heatmap")